In [1]:
import joblib
import json
from copy import copy
import importlib
import os
import sys
import numpy as np
import rpy2.robjects as robjects
import pandas as pd
from datetime import datetime
robjects.r('.sourceAlfunction()')
print(sys.path[0])
sys.path.insert(0, os.path.abspath(os.path.join(sys.path[0], 'NEXT/4EBaseMetal/code/utils')))
read_data = importlib.import_module("read_data")
construct_data = importlib.import_module("construct_data")
transform_data = importlib.import_module("transform_data")
normalize_feature = importlib.import_module("normalize_feature")

 

[1]
 "SinglePackageAl loaded. Attaching version 19.04.6"


[1]
 "Loaded AlphienData"


[1]
 "Loaded AlphienTechnicalAnalysis"


[1]
 "Loaded AlphienFundamental"


[1]
 "Loaded AlphienBaseMetals"


[1]
 "Loaded AlphienOption"


send2Log: Your working directory is: /home/chanmingwei
 





ImportError: No module named 'utils'

In [ ]:
def h1_function(date_range,ground_truth):
    h = 1
    time_series = None
    rgetSecurity = robjects.r('getSecurity')
    rindex = robjects.r('index')
    rgetGenOHLCV = robjects.r('getGenOHLCV')
    extra_data = None
    extra_index = None
    dates = date_range.split("::")
    start_date = dates[0].split("-")
    start_date = datetime(start_date[0],start_date[1],start_date[2])
    end_date = dates[1].split("-")
    end_date = datetime(end_date[0],end_date[1],end_date[2])
    if ground_truth == "LMAHDY Comdty":
        lag = 20
        norm_ex = "v1"
        norm_volume = "v1"
        norm_3m_spread = "v1"
        extra_data = robjects.r('getGenOHLCV("SI")[,6]')
#         print(extra_data)
        extra_index = rindex(extra_data)
    elif ground_truth == "LMPBDY Comdty":
        lag = 40
        norm_ex = "v1"
        norm_volume = "v1"
        norm_3m_spread = "v1"
        extra_data = robjects.r('getSecurityOHLCV("LMSNDS03 Comdty")[,6]')
        extra_index = rindex(extra_data)
    elif ground_truth == "LMNIDY Comdty":
        lag = 5
        norm_ex = "v1"
        norm_volume = "v1"
        norm_3m_spread = "v1"
        extra_data = robjects.r('getGenOHLCV("AKcl")[,3]')
        extra_index = rindex(extra_data)
#add the test function of the LMZSDY and LMSNDY
    elif ground_truth == "LMZSDY Comdty":
        lag = 20
        norm_ex = "v1"
        norm_volume = "v1"
        norm_3m_spread = "v1"
        extra_data = robjects.r('getGenOHLCV("AEcl")[,2]')
        extra_index = rindex(extra_data)
    elif ground_truth == "LMSNDY Comdty":
        lag = 40
        norm_ex = "v1"
        norm_volume = "v1"
        norm_3m_spread = "v1"
        extra_data = robjects.r('getGenOHLCV("RTcl")[,5]')
        extra_index = rindex(extra_data)
    elif ground_truth == "LMCADY Comdty":
        lag = 5
        norm_ex = "v1"
        norm_volume = "v1"
        norm_3m_spread = "v1"
        extra_data = rgetSecurity(securityName= "LMCADY Comdty", field = "Open Price")
        extra_index = rindex(extra_data)
    else:
        print("ground truth val out of bounds!")
        return None
    gt_data = rgetSecurity(securityName=ground_truth,zoom=date_range)
    gt_index = rindex(gt_data)
    gt_data_list = []
    gt_index_list = []
    extra_data_list = []
    extra_index_list = []
    for i in range(len(gt_index)):
        gt_index_list.append(gt_index[i])
    for i in range(len(gt_data)):
        gt_data_list.append(gt_data[i])
    for i in range(len(extra_index)):
        extra_index_list.append(extra_index[i])
    for i in range(len(extra_data)):
        extra_data_list.append(extra_data[i])
    start_date= min(gt_index_list[0],extra_index_list[0])
    end_date =max(gt_index_list[-1],extra_index_list[-1])
    if ground_truth == "LMAHDY Comdty":
        gt_dict={'time':gt_index_list,'LME_Al_Spot':gt_data_list}
        extra_dict = {'time':extra_index_list+86400*np.ones(len(extra_index_list)), 'COMEX_SI_lag1_OI':extra_data_list}
    elif ground_truth == "LMPBDY Comdty":
        gt_dict={'time':gt_index_list,'LME_Le_Spot':gt_data_list}
        extra_dict = {'time':extra_index_list, 'LME_Ti_OI':extra_data_list}
    elif ground_truth == "LMNIDY Comdty":
        gt_dict={'time':gt_index_list,'LME_Ni_Spot':gt_data_list}
        extra_dict = {'time':extra_index_list, 'DCE_AK_Low':extra_data_list}
    elif ground_truth == "LMZSDY Comdty":
        gt_dict={'time':gt_index_list,'LME_Zi_Spot':gt_data_list}
        extra_dict = {'time':extra_index_list, 'DCE_AE_High':extra_data_list}
    elif ground_truth == "LMSNDY Comdty":
        gt_dict={'time':gt_index_list,'LME_Ti_Spot':gt_data_list}
        extra_dict = {'time':extra_index_list, 'SHFE_RT_Volume':extra_data_list}
    elif ground_truth == "LMCADY Comdty":
        gt_dict={'time':gt_index_list,'LME_Co_Spot':gt_data_list}
        extra_dict = {'time':extra_index_list, 'LME_Co_Open':extra_data_list}
    else:
        print("ground truth val out of bounds!")
        return None
    gt_dataframe = pd.DataFrame(gt_dict)
    extra_dataframe = pd.DataFrame(extra_dict)
#     print(isinstance(extra_dataframe, pd.DataFrame))
    time_series = pd.DataFrame.merge(gt_dataframe,extra_dataframe,how ='outer', on = 'time')
    time_series.set_index('time')
#     print(time_series['time'])
    time_series = time_series.drop('time',axis = 1)
    

#     print(time_series)
    
    #Processing data
    time_series = read_data.process_missing_value_v3(time_series,np.min([lag,10]))
    org_cols = time_series.columns.values.tolist()
    print("Normalizing")

    #Normalize and generate technical indications
    norm_params = construct_data.normalize(time_series,vol_norm = norm_volume, spot_spread_norm=norm_3m_spread,ex_spread_norm = norm_ex)
    time_series = copy(norm_params["val"])
    del norm_params["val"]
    time_series = construct_data.technical_indication(time_series)
    cols = time_series.columns.values.tolist()
    for col in cols:
        if "_Volume" in col or "_OI" in col or "CNYUSD" in col:
            time_series = time_series.drop(col,axis = 1)
            org_cols.remove(col)
    curr = ""
    if ground_truth == "LMAHDY Comdty":
        curr = "Al"
    elif ground_truth == "LMPBDY Comdty":
        curr = "Le"
    elif ground_truth == "LMNIDY Comdty":
        curr = "Ni"
    elif ground_truth == "LMZSDY Comdty":
        curr = "Zi"
    elif ground_truth == "LMSNDY Comdty":
        curr = "Ti"
    elif ground_truth == "LMCADY Comdty":
        curr = "Co"
    else:
        print("ground truth val out of bounds!")
        return None

    norm_data = copy(normalize_feature.log_1d_return(time_series,org_cols))
    norm_data = read_data.process_missing_value_v3(norm_data,10)
    to_be_predicted = norm_data["LME_"+curr+"_Spot"]
#     if S > 1:
#         for i in range(S-1):
#             to_be_predicted = to_be_predicted + norm_data[gt_column].shift(-i-1)
    gt = (to_be_predicted > 0).shift(-1)

    start_index = 0
    if start_index < lag - 1:
        start_index = lag - 1
    end_index = len(norm_data)-1
    assert end_index >= lag - 1
#     time_series = pd.DataFrame.merge(gt_dataframe,extra_dataframe,how ='outer', on = 'time')
#     times = [datetime.fromtimestamp(time).strftime("%A, %B %d, %Y %I:%M:%S") for time in time_series['time'][norm_data.index[0]:norm_data.index[-1]]]
#     print((*times), sep = "\n")
    X_te, y_te = construct_data.construct(norm_data, gt, start_index, end_index, lag, "log_1d_return")
    X_te = transform_data.flatten(X_te)
    y_te = y_te*2 - 1
    print(len(y_te))
    #load model
    model = joblib.load("NEXT/LME_"+curr+"_Spot_h"+str(h)+"_n2.joblib")

    prediction = model.predict(X_te).reshape(X_te.shape[0],1)

    with open("NEXT/Nickel h"+str(h)+".csv","w") as out:
        out.write("Prediction,True Value\n")
        for i in range(X_te.shape[0]):
            # outputs a boolean value stating whether the prediction was an increase in spot price.
            out.write(str(prediction[i] == 1).strip("[ ").strip("]") + ","+ str(y_te[i] == 1).strip("[ ").strip("]") + "\n")

    total_no = prediction.shape[0]
    no_true = sum(np.equal(prediction,y_te))
    no_TT = sum(np.multiply(prediction+1,y_te+1))/4
    no_FF = sum(np.multiply(prediction - 1,y_te - 1))/4
    no_TF = -sum(np.multiply(prediction + 1,y_te - 1))/4
    no_FT = -sum(np.multiply(prediction - 1,y_te + 1))/4

    #some basic statistics, to identify whether our model is more prone to which type of error.
    print("Overall Accuracy:%d",no_true/total_no )
    print("TT:%d", no_TT)
    print("TF:%d", no_TF)
    print("FT:%d", no_FT)
    print("FF:%d", no_FF)
    return prediction



In [ ]:
# LMAHDY = h1_function("2016::","LMAHDY Comdty")
LMAHDY_test = h1_function("2016-01-06::2017-01-05","LMAHDY Comdty")

In [ ]:
LMAHDY_holdout = h1_function("2017-01-06::","LMAHDY Comdty")

In [ ]:
# LMCADY = h1_function("2016::","LMCADY Comdty")
LMCADY_test = h1_function("2016-01-06::2017-01-05","LMCADY Comdty")

In [ ]:
LMCADY_holdout = h1_function("2017-01-06::","LMCADY Comdty")

In [ ]:
# LMNIDY = h1_function("2016::","LMNIDY Comdty")
LMNIDY_test = h1_function("2016-01-06::2017-01-05","LMNIDY Comdty")

In [ ]:
LMNIDY_holdout = h1_function("2017-01-06::","LMNIDY Comdty")

In [ ]:
# LMPBDY = h1_function("2016::","LMPBDY Comdty")
LMPBDY_test = h1_function("2016-01-06::2017-01-05","LMPBDY Comdty")

In [ ]:
LMPBDY_holdout = h1_function("2017-01-06::","LMPBDY Comdty")

In [ ]:
# LMSNDY = h1_function("2016::","LMSNDY Comdty")
LMSNDY_test = h1_function("2016-01-06::2017-01-05","LMSNDY Comdty")

In [ ]:
LMSNDY_holdout = h1_function("2017-01-06::","LMSNDY Comdty")

In [ ]:
# LMZSDY = h1_function("2016::","LMZSDY Comdty")
LMZSDY = h1_function("2016-01-06::2017-01-05","LMZSDY Comdty")

In [ ]:
LMZSDY = h1_function("2017-01-06::","LMZSDY Comdty")